In [1]:
import pandas as pd
import pygsheets

In [2]:

#from datetime import date, time
DelDF = pd.read_csv('Deliveries.csv')
OldDelDF = pd.read_csv('DeliveriesOld.csv')
DaysDF = pd.read_csv('Days.csv')
WeeksDF = pd.read_csv('Weeks.csv')
gpsDF = pd.read_csv('GPS.csv')
rawDF = pd.read_csv('RawData.csv')

In [22]:

gc = pygsheets.authorize(service_file='file.json')
sh = gc.open('DashData')
for x in sh:
    print(x)
#wks = sh[7]
#df7 = wks.get_as_df()
#df7
sheets_OldDelDF    = sh[0].get_as_df()
sheets_DelDF       = sh[1].get_as_df()
sheets_DaysDF      = sh[2].get_as_df()
sheets_WeeksDF     = sh[3].get_as_df()
sheets_RawNamesDF  = sh[4].get_as_df()
sheets_GPSDF       = sh[5].get_as_df()

<Worksheet 'DeliveriesOld' index:0>
<Worksheet 'Deliveries' index:1>
<Worksheet 'Days' index:2>
<Worksheet 'Weeks' index:3>
<Worksheet 'RawNames' index:4>
<Worksheet 'GPS' index:5>
<Worksheet 'Lifetime Deliveries' index:6>
<Worksheet 'Import' index:7>
<Worksheet 'Days Old' index:8>
<Worksheet 'Weeks Old' index:9>


In [4]:
#For every attribute x from columns, replace $ and , with empty, then convert to float
#Should we do for index, row then df.at[index, x] = float()
#Or just pop it from the for loop?

def dollars_to_floats(df, columns):
    for x in columns:
        df[x] = df[x].str.replace('$', '')
        df[x] = df[x].str.replace(',', '')
        df[x] = df[x].astype(float)

        
#For every attribute x from columns, for every row convert hh:mm string to double

def time_to_floats(df, columns):
    for x in columns:
        for index, row in df.iterrows():
            active = row[x]
            activeH = float(active[:active.index(":")])
            activeM = round(float(active[active.index(":")+1:active.index(":")+3])/60, 2)
            active = activeH + activeM
            df.at[index, x] = active
        df[x] = df[x].astype(float)
            
            
#This obviously only works for 2023
#Might want to homogenize to columns just in case it shows up later but for now only use is 1 column

def date_to_timestamp(df, column):
    for index, row in df.iterrows():
        df.at[index, column] = pd.Timestamp('2023/' + row[column])
        
        
#This only works for 2023, but also orders that end on the same date as shift start
#So if an order finishes at 1am it wont work
#Need to update with conditional like if end is AM start is PM then date +=1

def time_to_timestamp(df, DateCol, TimeCols):
    for index, row in df.iterrows():
        df.at[index, TimeCols[0]] = pd.Timestamp('2023/' + row[DateCol] + " " + row[TimeCols[0]])
        df.at[index, TimeCols[1]] = pd.Timestamp('2023/' + row[DateCol] + " " + row[TimeCols[1]])

In [23]:
frames = [OldDelDF, DelDF]
result = pd.concat(frames)
#We need to reset the index after the combination of v1/v2, and v3
result.reset_index(drop=True, inplace=True)

#DelDF is v3
#results is all deliveries
#Those are the only 2 we need


sheets_frames = [sheets_OldDelDF, sheets_DelDF]
sheets_result = pd.concat(sheets_frames)
sheets_result.reset_index(drop=True, inplace=True)

In [25]:
sheets_result.dtypes

ID                  int64
Date               object
Restaurant Name    object
Total              object
Pay                object
Tip                object
Peak Bonus         object
Stacked            object
Version             int64
Start Time         object
Miles              object
End Time           object
Est. Duration      object
dtype: object

In [6]:
result.dtypes

ID                  int64
Date               object
Restaurant Name    object
Total              object
Pay                object
Tip                object
Peak Bonus         object
Stacked            object
Version             int64
Start Time         object
Miles              object
End Time           object
Est. Duration      object
dtype: object

In [26]:
dollars_to_floats(sheets_result, ["Total", "Pay", "Tip", "Peak Bonus"])

/tmp/ipykernel_9671/691726310.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[x] = df[x].str.replace('$', '')


In [27]:
#Doodle to get the percentage of orders that tip
numUntipped = (sheets_result["Tip"] == 0.0).sum()
numTipped = (sheets_result["Tip"] != 0.0).sum()
percTipped = round((numTipped/(numUntipped+numTipped)), 3)
print(percTipped*100, "% of customers tip")

89.7 % of customers tip


In [9]:
# GPS DOODLE HERE
# Initial doodle to help with address entry
# Need to rework to handle both CSVs now
#gpsDF1 = gpsDF[["ID", "Date", "Restaurant Name", "RAW DATA"]].copy()
#gpsDF2 = gpsDF1[["Restaurant Name", "RAW DATA"]].copy()
#gpsDF2 = gpsDF2.drop_duplicates(subset=["Restaurant Name", "RAW DATA"])
#gpsDF2 = gpsDF2.sort_values(by="Restaurant Name", ascending=True)
#gpsDF2.to_csv('gpsDF2.csv')

In [28]:
tempDF = sheets_result.copy()
tempDF = tempDF[sheets_result["Restaurant Name"].duplicated(keep=False) == True]
tempDF = tempDF.groupby(["Restaurant Name"]).agg({"Pay": "mean", "Tip": "mean"})
tempDF = tempDF.round({'Pay': 2, 'Tip': 2})
tempDF.sort_values("Tip", ascending=False)

,Pay,Tip
Restaurant Name,,
Albertsons,4.58,12.39
Hunter Steakhouse,10.00,11.50
Chin's Szechwan,5.88,9.25
Mangia e Bevi,6.25,9.00
Killer Pizza From Mars,4.12,7.80
...,...,...
Lollicup Fresh,4.25,1.82
Carl's Jr.,4.38,1.50
Sizzler,12.75,1.25


In [30]:
#Overall hourly rate, Total from TOTALS / Dash
#Hardcoded index for Totals, should look for row where dates == "TOTALS" 

#Find row where Dates == "Totals"
#tempDF that row
#dollars_to_floats
#time_to_floats
#hourlyRate = tempDF["Total"]/tempDF["Dash"]
#print("$", hourlyRate)

tempDF = sheets_WeeksDF.copy()
#double bracket on iloc returns a df instead of series
tempDF = tempDF.iloc[[-1]]
dollars_to_floats(tempDF, ["Total"])
time_to_floats(tempDF, ["Dash"])
hourlyRate = tempDF["Total"]/tempDF["Dash"]
print(tempDF["Total"])
print(tempDF["Dash"])
float(tempDF["Dash"])
print("$", hourlyRate)

14    7112.19
Name: Total, dtype: float64
14    381.7
Name: Dash, dtype: float64
$ 14    18.632932
dtype: float64


/tmp/ipykernel_9671/691726310.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[x] = df[x].str.replace('$', '')


In [12]:
#So float(var) converts, but series.astype(float) is for series in df

In [31]:
#Doodle to 
#convert $#.## to float and hh:mm:ss to float, 
#then make the pre total to hours worked ratio, then sort and show

print(sheets_WeeksDF.dtypes)

dollars_to_floats(sheets_WeeksDF, ["Pre Total"])
time_to_floats(sheets_WeeksDF, ["Active"])

sheets_WeeksDF["pretotalactiveratio"] = sheets_WeeksDF["Pre Total"]/sheets_WeeksDF["Active"]
sheets_WeeksDF.sort_values(by=["pretotalactiveratio"])
#pre total is the money before adjustment, its just base pay + tip 

#Trying to maximize total, so I guess the question really is which factor or ratio of factors
#contributes the most

Dates         object
Base          object
Tip           object
Adj pay       object
Other         object
Pre Total     object
Total         object
Active        object
Dash          object
Deliveries     int64
Version       object
dtype: object


/tmp/ipykernel_9671/691726310.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[x] = df[x].str.replace('$', '')


,Dates,Base,Tip,Adj pay,Other,Pre Total,Total,Active,Dash,Deliveries,Version,pretotalactiveratio
14,TOTALS:,"$3,544.75","$2,714.44",$844.13,$8.87,6259.19,"$7,112.19",263.05,381:42:00,674,-,23.794678
2,4/10 - 4/16,$130.75,$96.13,$61.07,$0.00,226.88,$287.95,8.63,11:10:00,28,1,26.289687
13,7/3 - 7/9,$114.25,$105.10,$54.99,$7.75,219.35,$282.09,7.83,10:10:00,24,3,28.014049
4,4/24 - 4/30,$430.00,$299.73,$138.76,$2.62,729.73,$871.11,25.50,41:10:00,84,1,28.616863
12,6/19 - 6/25,$637.25,$475.09,$240.02,$6.25,1112.34,"$1,358.61",38.60,51:14:00,124,3,28.817098
0,3/27 - 4/2,$108.50,$88.00,$40.96,$0.00,196.50,$237.46,6.70,10:55:00,22,1,29.328358
3,4/17 - 4/23,$382.50,$248.41,$105.05,$0.00,630.91,$735.96,21.33,32:18:00,68,1,29.578528
7,5/15 - 5/21,$303.75,$239.25,$99.27,$0.00,543.00,$642.27,17.53,25:24:00,64,2,30.975471
10,6/5 - 6/11,$197.25,$185.09,$65.38,$0.00,382.34,$447.72,11.77,15:45:00,41,3,32.484282
6,5/8 - 5/14,$758.25,$567.84,$155.48,$6.25,1326.09,"$1,487.82",40.58,58:08:00,138,2,32.678413


In [32]:
#Dollars Earned vs Miles Driven doodle
#$ to float
#drop non numeric Miles values, then convert column to float
#agg sum Total and Miles, by restaurant name (then round)
#create new column as Total/Miles
#sort low to high
tempDF = sheets_DelDF.copy()
dollars_to_floats(tempDF, ["Total"])
tempDF = tempDF[tempDF.Miles!="-"]
tempDF["Miles"] = tempDF["Miles"].astype(float)
tempDF = tempDF.groupby(["Restaurant Name"]).agg({"Total": "sum", "Miles": "sum"})
tempDF = tempDF.round({'Pay': 2, 'Tip': 2})
tempDF["DollarMilesRatio"] = tempDF["Total"]/tempDF["Miles"]
tempDF.sort_values(by="DollarMilesRatio")

/tmp/ipykernel_9671/691726310.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[x] = df[x].str.replace('$', '')


,Total,Miles,DollarMilesRatio
Restaurant Name,,,
Yukiya Sushi,8.25,10.8,0.763889
Pedro's Tacos,7.00,7.9,0.886076
Papa Johns,5.00,4.9,1.020408
Little Caesars,8.25,7.9,1.044304
Urban Plates,21.50,20.4,1.053922
...,...,...,...
Albertsons,34.17,8.8,3.882955
Graziano's Piza,9.50,2.3,4.130435
Ohana Hawaiian BBQ,8.25,1.8,4.583333


In [33]:
sheets_DaysDF.max() #would be useful with more attributes converted to float 

/tmp/ipykernel_9671/842620842.py:1: FutureWarning: The default value of numeric_only in DataFrame.max is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  sheets_DaysDF.max() #would be useful with more attributes converted to float


Date                    7/8
Total                $86.13
Base                 $99.75
Tip                  $94.03
Start (24)            18:58
End (24)               2:32
Active time (h:m)      9:40
Dash time (h:m)        9:48
Offers                   32
Deliveries               33
Version                   3
dtype: object

In [34]:
#Avg tip vs Avg tip excluding zero tip orders

TipAvg = sheets_result["Tip"].mean()
print(TipAvg)
TipAvg2 = sheets_result[sheets_result["Tip"] !=0]["Tip"].mean()
print(TipAvg2)

4.050644796380091
4.51547288776797


In [35]:
#CONVERT STRING OF DATE TO TIMESTAMP, THEN TIMESTAMP TO DATETIME
date_to_timestamp(sheets_DaysDF, "Date")
sheets_DaysDF["Date"] = pd.to_datetime(sheets_DaysDF["Date"])

In [36]:
#Adds column to DaysDF for the day of the week (Monday, Thursday, etc.)
sheets_DaysDF["Day of Week"] = sheets_DaysDF["Date"].dt.day_name()

In [37]:
time_to_floats(sheets_DaysDF, ["Dash time (h:m)"])
dollars_to_floats(sheets_DaysDF, ["Total", "Base", "Tip"])

/tmp/ipykernel_9671/691726310.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[x] = df[x].str.replace('$', '')


In [39]:
sheets_DaysDF = sheets_DaysDF.groupby(["Day of Week"]).agg(
    {"Dash time (h:m)": "sum", "Total": "sum", "Tip": "sum", "Deliveries ": "sum"})
sheets_DaysDF = sheets_DaysDF.round({'Dash time (h:m)': 2, 'Total': 2, "Tip": 2})
sheets_DaysDF["$/hr"] = sheets_DaysDF["Total"]/sheets_DaysDF["Dash time (h:m)"]
sheets_DaysDF["AvgTip"] = sheets_DaysDF["Tip"]/sheets_DaysDF["Deliveries "]
display(sheets_DaysDF)

,Dash time (h:m),Total,Tip,Deliveries,$/hr,AvgTip
Day of Week,,,,,,
Friday,112.52,2491.97,1133.97,265,22.146907,4.279132
Monday,17.85,377.47,158.97,41,21.146779,3.877317
Saturday,91.91,2064.22,904.22,215,22.459145,4.205674
Sunday,37.63,882.78,408.03,90,23.459474,4.533667
Thursday,63.46,1326.46,559.71,151,20.902301,3.706689
Wednesday,57.80,1047.37,415.87,121,18.120588,3.436942
